### 由于这部分代码在Sklearn中没有相关库，目前参考的[博客](http://blog.csdn.net/u010454729/article/details/49078505)
[参考1](https://www.cnblogs.com/90zeng/p/apriori.html)

[参考2](https://blog.csdn.net/wenyusuran/article/details/40736141)

[参考3](https://blog.csdn.net/u010454729/article/details/49078505?spm=a2c4e.11153940.blogcont70455.16.19bc5bfdBuH32L)

[扩展](https://www.cnblogs.com/lsqin/p/9342926.html)

### [代码解析](https://www.cnblogs.com/90zeng/p/apriori.html)

In [2]:
# 示例数据
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]
# 创建每个项集元素长度为一的 
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))#use frozen set so we
                            #can use it as a key in a dict    

def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt: 
                    ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(list(D)))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList         

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print((freqSet-conseq,'-->',conseq,'conf:',conf))
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)
            
def pntRules(ruleList, itemMeaning):
    for ruleTup in ruleList:
        for item in ruleTup[0]:
            print((itemMeaning[item]))
        print("           -------->")
        for item in ruleTup[1]:
            print((itemMeaning[item]))
        print(("confidence: %f" % ruleTup[2]))
        print()       #print a blank line

### 测试结果

In [26]:
if __name__ == '__main__':
    # 导入数据集
    myDat = loadDataSet()
    # 构建第一个候选项集列表C1
    C1 = createC1( myDat )
    
    # 构建集合表示的数据集 D
    D = list(map( set, myDat ))
    # 选择出支持度不小于0.5 的项集作为频繁项集
    L, suppData = scanD( D, C1, 0.5 )
   
    
    

频繁项集L： [frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]
所有候选项集的支持度信息： {frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75}


### 单个数据项结果展示

In [3]:
# 导入数据集
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [4]:
# 构建候选项集合C1
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [5]:
# 转用集合表示数据集
D = list(map(set, dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [6]:
# 过滤掉不满足最小支持度的项集，此处使用0.5 为最小支持度
L1, suppData = scanD(D, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [7]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75}

### 多个数据项结果展示

In [9]:
L, suppData = apriori(dataSet)
# L为最小支持度默认为0.5 的频繁项集列表
print("频繁项集L：", L)

频繁项集L： [[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})], [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})], [frozenset({2, 3, 5})], []]


In [10]:
print("所有候选项集的支持度信息：", suppData)

所有候选项集的支持度信息： {frozenset({1}): 0.5, frozenset({3}): 0.75, frozenset({4}): 0.25, frozenset({2}): 0.75, frozenset({5}): 0.75, frozenset({1, 3}): 0.5, frozenset({2, 5}): 0.75, frozenset({3, 5}): 0.5, frozenset({2, 3}): 0.5, frozenset({1, 5}): 0.25, frozenset({1, 2}): 0.25, frozenset({2, 3, 5}): 0.5}


#### 具体的值

In [11]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [12]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [13]:
L[2]

[frozenset({2, 3, 5})]

In [17]:
# aprioriGen 函数工作流程
aprioriGen(L[0], 2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

#### 70%支持度结果

In [15]:
L, suppData = apriori(dataSet, minSupport=0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

### 规则展示

#### 0.7置信度

In [17]:
dataSet = loadDataSet()
# 先生成一个支持度为0.5的频繁项集合
L, suppData = apriori(dataSet, minSupport=0.5)
# 生成最小置信度为0.7的规则
rules = generateRules(L, suppData, minConf=0.7)
# rules
#给出三条规则，观察知前两条可以互换前件和后件，但第三条不行

(frozenset({5}), '-->', frozenset({2}), 'conf:', 1.0)
(frozenset({2}), '-->', frozenset({5}), 'conf:', 1.0)
(frozenset({1}), '-->', frozenset({3}), 'conf:', 1.0)


#### 降低为0.5置信度

In [8]:
rules = generateRules(L, suppData, minConf=0.5)
# 一旦降低可信度阈值，就可以获得更多的规则。

(frozenset({3}), '-->', frozenset({2}), 'conf:', 0.6666666666666666)
(frozenset({2}), '-->', frozenset({3}), 'conf:', 0.6666666666666666)
(frozenset({5}), '-->', frozenset({3}), 'conf:', 0.6666666666666666)
(frozenset({3}), '-->', frozenset({5}), 'conf:', 0.6666666666666666)
(frozenset({5}), '-->', frozenset({2}), 'conf:', 1.0)
(frozenset({2}), '-->', frozenset({5}), 'conf:', 1.0)
(frozenset({3}), '-->', frozenset({1}), 'conf:', 0.6666666666666666)
(frozenset({1}), '-->', frozenset({3}), 'conf:', 1.0)
(frozenset({5}), '-->', frozenset({2, 3}), 'conf:', 0.6666666666666666)
(frozenset({3}), '-->', frozenset({2, 5}), 'conf:', 0.6666666666666666)
(frozenset({2}), '-->', frozenset({3, 5}), 'conf:', 0.6666666666666666)


### 毒蘑菇示例

In [19]:
cd C:\Users\wanzheng\Desktop\MLiA_SourceCode\machinelearninginaction\Ch11

C:\Users\wanzheng\Desktop\MLiA_SourceCode\machinelearninginaction\Ch11


In [20]:
'''mushroom.dat 前几行如下：

1 3 9 13 23 25 34 36 38 40 52 54 59 63 67 76 85 86 90 93 98 107 113 
2 3 9 14 23 26 34 36 39 40 52 55 59 63 67 76 85 86 90 93 99 108 114 
2 4 9 15 23 27 34 36 39 41 52 55 59 63 67 76 85 86 90 93 99 108 115 
1 3 10 15 23 25 34 36 38 41 52 54 59 63 67 76 85 86 90 93 98 107 113 
2 3 9 16 24 28 34 37 39 40 53 54 59 63 67 76 85 86 90 94 99 109 114 


第一个特征表示有毒或者可食用,如果有毒，则赋值为2。如果可食用则值为1
下一个特征是蘑菇伞的形状，共有6种可能，分别用3-8来表示。其他特征解释略过。

'''

'mushroom.dat 前几行如下：\n\n1 3 9 13 23 25 34 36 38 40 52 54 59 63 67 76 85 86 90 93 98 107 113 \n2 3 9 14 23 26 34 36 39 40 52 55 59 63 67 76 85 86 90 93 99 108 114 \n2 4 9 15 23 27 34 36 39 41 52 55 59 63 67 76 85 86 90 93 99 108 115 \n1 3 10 15 23 25 34 36 38 41 52 54 59 63 67 76 85 86 90 93 98 107 113 \n2 3 9 16 24 28 34 37 39 40 53 54 59 63 67 76 85 86 90 94 99 109 114 \n\n\n第一个特征表示有毒或者可食用,如果有毒，则赋值为2。如果可食用则值为1\n下一个特征是蘑菇伞的形状，共有6种可能，分别用3-8来表示。其他特征解释略过。\n\n'

In [21]:
# 加载数据集
mushDatSet = [line.split() for line in open('mushroom.dat').readlines()]
# 运行Apriori算法
L, suppData = apriori(mushDatSet, minSupport=0.3)

In [22]:
L

[[frozenset({'58'}),
  frozenset({'56'}),
  frozenset({'116'}),
  frozenset({'6'}),
  frozenset({'110'}),
  frozenset({'94'}),
  frozenset({'53'}),
  frozenset({'28'}),
  frozenset({'24'}),
  frozenset({'10'}),
  frozenset({'39'}),
  frozenset({'2'}),
  frozenset({'93'}),
  frozenset({'90'}),
  frozenset({'9'}),
  frozenset({'86'}),
  frozenset({'85'}),
  frozenset({'76'}),
  frozenset({'67'}),
  frozenset({'63'}),
  frozenset({'59'}),
  frozenset({'52'}),
  frozenset({'38'}),
  frozenset({'36'}),
  frozenset({'34'}),
  frozenset({'3'}),
  frozenset({'23'}),
  frozenset({'1'})],
 [frozenset({'58', '85'}),
  frozenset({'1', '24'}),
  frozenset({'36', '53'}),
  frozenset({'116', '36'}),
  frozenset({'36', '56'}),
  frozenset({'39', '56'}),
  frozenset({'110', '24'}),
  frozenset({'110', '53'}),
  frozenset({'116', '34'}),
  frozenset({'116', '85'}),
  frozenset({'116', '86'}),
  frozenset({'116', '90'}),
  frozenset({'34', '56'}),
  frozenset({'56', '85'}),
  frozenset({'56', '86'}),
  f

In [7]:
# 在结果中搜索含有毒特征值为2的频繁项集：
for item in L[1]:
    if '2' in item :
        print(item)

frozenset({'2', '28'})
frozenset({'2', '53'})
frozenset({'2', '23'})
frozenset({'2', '34'})
frozenset({'2', '36'})
frozenset({'2', '59'})
frozenset({'2', '63'})
frozenset({'2', '67'})
frozenset({'2', '76'})
frozenset({'2', '85'})
frozenset({'2', '86'})
frozenset({'2', '90'})
frozenset({'2', '93'})
frozenset({'2', '39'})


In [23]:
for item in L[3]:
    if '2' in item :
        print(item)

frozenset({'2', '28', '34', '85'})
frozenset({'59', '2', '28', '34'})
frozenset({'59', '2', '63', '28'})
frozenset({'59', '2', '28', '85'})
frozenset({'59', '2', '28', '90'})
frozenset({'59', '39', '2', '28'})
frozenset({'63', '2', '28', '34'})
frozenset({'63', '2', '28', '85'})
frozenset({'86', '53', '2', '28'})
frozenset({'86', '39', '2', '28'})
frozenset({'86', '2', '28', '34'})
frozenset({'86', '59', '2', '28'})
frozenset({'86', '63', '2', '28'})
frozenset({'86', '2', '28', '85'})
frozenset({'86', '2', '28', '90'})
frozenset({'2', '28', '90', '34'})
frozenset({'2', '28', '90', '85'})
frozenset({'39', '2', '28', '90'})
frozenset({'53', '2', '28', '90'})
frozenset({'39', '2', '28', '34'})
frozenset({'39', '2', '63', '28'})
frozenset({'39', '2', '28', '85'})
frozenset({'53', '2', '28', '34'})
frozenset({'53', '2', '28', '85'})
frozenset({'39', '53', '2', '28'})
frozenset({'53', '2', '34', '85'})
frozenset({'53', '39', '2', '34'})
frozenset({'86', '53', '39', '2'})
frozenset({'86', '53